# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
### <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
**Primer Examen**

**Fecha**: 14 de Marzo del 2025

**Nombre del estudiante**:

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Exam-1-Yael-Rodriguez") \
    .master("spark://8b1848b8768b:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 13:37:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [52]:
from equipo.spark_utils import SparkUtils

columns_info = [
    ("employee_id", "string"), 
    ("employee_info", "string")
]

employee_schema = SparkUtils.generate_schema(columns_info)

employee_df = spark.read \
                .schema(employee_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/employees.csv")

employee_df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- employee_info: string (nullable = true)



In [53]:
from equipo.spark_utils import SparkUtils

columns_info = [
    ("department_id", "string"), 
    ("department_name", "string"),
    ("location", "string")
]

departments_schema = SparkUtils.generate_schema(columns_info)

departments_df = spark.read \
                .schema(departments_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/exam_P2025_ESI3914B/departments.csv")

departments_df.printSchema()

root
 |-- department_id: string (nullable = true)
 |-- department_name: string (nullable = true)
 |-- location: string (nullable = true)



In [54]:
from pyspark.sql.functions import get_json_object
employee_df = employee_df.withColumn("name", get_json_object(employee_df.employee_info, '$.name')) \
    .withColumn("department_id", get_json_object(employee_df.employee_info, '$.department_id')) \
    .withColumn("salary", get_json_object(employee_df.employee_info, '$.salary')) \
    .withColumn("hire_date", get_json_object(employee_df.employee_info, '$.hire_date')) 
employee_df = employee_df.withColumn("department_id", employee_df.department_id.cast('integer')) \
    .withColumn("salary", employee_df.salary.cast('double')) \
    .withColumn("hire_date", employee_df.hire_date.cast('date'))
employee_df.show(n=5)
employee_df.printSchema()

+-----------+--------------------+---------+-------------+---------+----------+
|employee_id|       employee_info|     name|department_id|   salary| hire_date|
+-----------+--------------------+---------+-------------+---------+----------+
|          1|{'name': 'Caitlyn...|  Caitlyn|          103|115959.78|2002-06-10|
|          2|{'name': 'Rachel'...|   Rachel|          104|100820.16|2009-07-01|
|          3|{'name': 'Carrie'...|   Carrie|          105|114421.44|1998-12-10|
|          4|{'name': 'Renee',...|    Renee|          104| 54688.13|1995-03-17|
|          5|{'name': 'Gabriel...|Gabriella|          109|106267.03|1995-02-09|
+-----------+--------------------+---------+-------------+---------+----------+
only showing top 5 rows

root
 |-- employee_id: string (nullable = true)
 |-- employee_info: string (nullable = true)
 |-- name: string (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- salary: double (nullable = true)
 |-- hire_date: date (nullable = true)



In [55]:
from pyspark.sql.functions import when
employee_df = employee_df.withColumn('salary_category', when(employee_df['salary'] > 55000, "High").otherwise("Low"))
employee_df.show(n=5)

+-----------+--------------------+---------+-------------+---------+----------+---------------+
|employee_id|       employee_info|     name|department_id|   salary| hire_date|salary_category|
+-----------+--------------------+---------+-------------+---------+----------+---------------+
|          1|{'name': 'Caitlyn...|  Caitlyn|          103|115959.78|2002-06-10|           High|
|          2|{'name': 'Rachel'...|   Rachel|          104|100820.16|2009-07-01|           High|
|          3|{'name': 'Carrie'...|   Carrie|          105|114421.44|1998-12-10|           High|
|          4|{'name': 'Renee',...|    Renee|          104| 54688.13|1995-03-17|            Low|
|          5|{'name': 'Gabriel...|Gabriella|          109|106267.03|1995-02-09|           High|
+-----------+--------------------+---------+-------------+---------+----------+---------------+
only showing top 5 rows



In [56]:
low_df = employee_df.filter(employee_df["salary_category"] == 'Low')
high_df = employee_df.filter(employee_df["salary_category"] == 'High')

low_df.show(n=3)
high_df.show(n=3)

+-----------+--------------------+--------+-------------+--------+----------+---------------+
|employee_id|       employee_info|    name|department_id|  salary| hire_date|salary_category|
+-----------+--------------------+--------+-------------+--------+----------+---------------+
|          4|{'name': 'Renee',...|   Renee|          104|54688.13|1995-03-17|            Low|
|          7|{'name': 'Jonatha...|Jonathan|          102|39323.42|2012-06-30|            Low|
|         13|{'name': 'Lisa', ...|    Lisa|          104|36032.49|2019-05-16|            Low|
+-----------+--------------------+--------+-------------+--------+----------+---------------+
only showing top 3 rows

+-----------+--------------------+-------+-------------+---------+----------+---------------+
|employee_id|       employee_info|   name|department_id|   salary| hire_date|salary_category|
+-----------+--------------------+-------+-------------+---------+----------+---------------+
|          1|{'name': 'Caitlyn...|C

In [57]:
grouped_low_df = low_df.select('salary', 'department_id' )
grouped_low_df = grouped_low_df.groupBy('department_id').avg('salary').withColumnRenamed('avg(salary)', 'avg_salary')

grouped_high_df = high_df.select('salary', 'department_id' )
grouped_high_df = grouped_high_df.groupBy('department_id').avg('salary').withColumnRenamed('avg(salary)', 'avg_salary')

grouped_low_df.show(n=3)
grouped_high_df.show(n=3)

+-------------+-----------------+
|department_id|       avg_salary|
+-------------+-----------------+
|          108|41426.43521126761|
|          101|41751.64784810126|
|          103|41150.40277777778|
+-------------+-----------------+
only showing top 3 rows

+-------------+------------------+
|department_id|        avg_salary|
+-------------+------------------+
|          108|  98714.3003086419|
|          101|104999.43191489363|
|          103|100839.65275449108|
+-------------+------------------+
only showing top 3 rows



In [58]:
low_df.createOrReplaceTempView("low")
high_df.createOrReplaceTempView("high")

spark.sql("SELECT employee_id, name, salary FROM low ORDER BY salary DESC ").show(5)
spark.sql("SELECT employee_id, name, salary FROM high ORDER BY salary DESC ").show(5)

+-----------+-----+--------+
|employee_id| name|  salary|
+-----------+-----+--------+
|       3472|Linda|54993.53|
|       2545|Tammy|54991.71|
|        382|Aaron|54989.45|
|       2153|Craig| 54945.2|
|       3024|Aaron| 54937.3|
+-----------+-----+--------+
only showing top 5 rows



+-----------+---------+---------+
|employee_id|     name|   salary|
+-----------+---------+---------+
|       1778|Gabriella|149989.73|
|       3621|Katherine| 149979.3|
|        346|     Ryan| 149963.1|
|       3807|  Caitlyn|149956.54|
|       3050|     Mark|149915.56|
+-----------+---------+---------+
only showing top 5 rows



In [59]:
from pyspark.sql.functions import year
employee_df = employee_df.withColumn('years', 2025 - year(employee_df['hire_date'])).orderBy(['years'], ascending = [False])
years = employee_df.select('years').collect()[0]

most_years = employee_df.filter(employee_df['years'] == years.years)
most_years.show(n=5)

print("Total =", most_years.count())


+-----------+--------------------+-----+-------------+---------+----------+---------------+-----+
|employee_id|       employee_info| name|department_id|   salary| hire_date|salary_category|years|
+-----------+--------------------+-----+-------------+---------+----------+---------------+-----+
|         59|{'name': 'Ana', '...|  Ana|          104| 85736.81|1988-12-31|           High|   37|
|        103|{'name': 'Seth', ...| Seth|          101|106867.83|1988-08-29|           High|   37|
|        120|{'name': 'Megan',...|Megan|          107| 56472.42|1988-04-03|           High|   37|
|        159|{'name': 'Sarah',...|Sarah|          103| 53202.89|1988-04-14|            Low|   37|
|        182|{'name': 'Mark', ...| Mark|          103|122033.27|1988-11-20|           High|   37|
+-----------+--------------------+-----+-------------+---------+----------+---------------+-----+
only showing top 5 rows

Total = 88
